In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("/content/MaunaLoaDailyTemps-1.csv")
df

,DATE,MinTemp,MaxTemp,AvgTemp,Sunrise,Sunset
0,1/1/2014,33.0,46.0,40.0,657,1756
1,1/2/2014,35.0,50.0,43.0,657,1756
2,1/3/2014,36.0,45.0,41.0,657,1757
3,1/4/2014,32.0,41.0,37.0,658,1757
4,1/5/2014,24.0,38.0,31.0,658,1758
...,...,...,...,...,...,...
1820,12/26/2018,35.0,45.0,40.0,654,1752
1821,12/27/2018,33.0,44.0,39.0,655,1752
1822,12/28/2018,33.0,47.0,40.0,655,1753
1823,12/29/2018,36.0,47.0,42.0,655,1753


In [12]:
df.columns

Index(['DATE', 'MinTemp', 'MaxTemp', 'AvgTemp', 'Sunrise', 'Sunset'], dtype='object')

In [19]:
df.corr()

<ipython-input-19-2f6f6606aa2c>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,MinTemp,MaxTemp,AvgTemp,Sunrise,Sunset
MinTemp,1.000000,0.756076,0.917454,-0.426621,0.300287
MaxTemp,0.756076,1.000000,0.951036,-0.374763,0.275688
AvgTemp,0.917454,0.951036,1.000000,-0.422591,0.304507
Sunrise,-0.426621,-0.374763,-0.422591,1.000000,-0.739530
Sunset,0.300287,0.275688,0.304507,-0.739530,1.000000


In [17]:
df.isnull().sum()

DATE       0
MinTemp    4
MaxTemp    4
AvgTemp    4
Sunrise    0
Sunset     0
dtype: int64

In [18]:
df.fillna(value=df.median(),inplace=True)

<ipython-input-18-10cf8360e98e>:1: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.fillna(value=df.median(),inplace=True)


In [3]:
df[df.AvgTemp > 50.0].shape[0]

331

In [11]:
df[df.MaxTemp == df.MaxTemp.max()].DATE

533    6/18/2015
914     7/3/2016
Name: DATE, dtype: object

In [31]:
from statsmodels.tsa.arima.model import ARIMA

model = ARIMA(df['AvgTemp'],order=(1,0,5))
model_fit = model.fit()
print(model_fit.bic)
model = ARIMA(df['AvgTemp'],order=(3,0,2))
model_fit = model.fit()
print(model_fit.bic)
model = ARIMA(df['AvgTemp'],order=(2,0,5))
model_fit = model.fit()
print(model_fit.bic)
model = ARIMA(df['AvgTemp'],order=(0,0,5))
model_fit = model.fit()
print(model_fit.bic)

8363.524744227609


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


8356.133476059491
8369.774778832116
8488.3130268686


In [39]:
model = ARIMA(df['AvgTemp'],order=(3,0,2))
model_fit = model.fit()
n_forecast = 20
predictions = model_fit.predict(start=len(df), end=len(df)+n_forecast-1, typ='levels')  # Replace n_forecast with the number of forecast steps

# Calculate Mean Absolute Error (MAE)
actual_values = df['AvgTemp'].iloc[-n_forecast:]  # Actual values corresponding to the forecast period


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [61]:
mae = np.mean(np.abs(predictions - actual_values))
mae

2.8007860447486044

In [64]:
ar_params = model_fit.arparams
ar_params.mean()

0.33206077992394833